In [ ]:
import pandas as pd
import heapq

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Beispiel-Koordinatenliste mit Längen- und Breitengraden
coordinates = [
    [40.7128, -74.0060],  # New York
    [34.0522, -118.2437],  # Los Angeles
    [51.5074, -0.1278],    # London
    [48.8566, 2.3522],     # Paris
    [37.7749, -122.4194]   # San Francisco
]

# Anzahl der nächsten Nachbarn, die Sie suchen möchten
k = 1

# Erstellen des k-NN-Modells
knn_model = NearestNeighbors(n_neighbors=k)
knn_model.fit(coordinates)

# Wählen Sie eine Koordinate, für die Sie den nächsten Nachbarn finden möchten
query_coordinate = [38.8951, -77.0364]  # Washington, DC

# Finden Sie die nächsten Nachbarn der ausgewählten Koordinate
distances, indices = knn_model.kneighbors([query_coordinate])

# Zeigen Sie die Informationen über den nächsten Nachbarn an
nearest_neighbor_index = indices[0][0]
nearest_neighbor_distance = distances[0][0]
nearest_neighbor = coordinates[nearest_neighbor_index]

print("Die nächste Koordinate zu", query_coordinate, "ist", nearest_neighbor)
print("Entfernung:", nearest_neighbor_distance)

In [ ]:
import numpy as np
import folium
from sklearn.neighbors import BallTree
from IPython.display import display

# Beispiel-Ladestationen mit Längen- und Breitengraden
charging_stations = [
    {"name": "Station 1", "latitude": 52.5200, "longitude": 13.4050},
    {"name": "Station 2", "latitude": 48.8566, "longitude": 2.3522},
    # Weitere Ladestationen hier...
]

# Wählen Sie einen Punkt (z.B. [38.8951, -77.0364]) aus, für den Sie die nächstgelegene Ladestation finden möchten
query_point = [38.8951, -77.0364]

# Koordinaten der Ladestationen
station_coords = np.array([(station["latitude"], station["longitude"]) for station in charging_stations])

# Erstellen des BallTree-Index für die Ladestationen
tree = BallTree(station_coords, leaf_size=2)

# Finden der nächstgelegenen Ladestation mit k-NN
distances, indices = tree.query([query_point], k=1)

# Index der nächsten Ladestation finden
nearest_station_index = indices[0][0]

# Koordinaten der nächsten Ladestation
nearest_station_lat = charging_stations[nearest_station_index]["latitude"]
nearest_station_long = charging_stations[nearest_station_index]["longitude"]

# Karte von Deutschland mit Folium erstellen
map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
m = folium.Map(location=map_center, zoom_start=6)

# Marker für Ladestationen hinzufügen
for station in charging_stations:
    popup_content = f"<b>Name:</b> {station['name']}"
    folium.Marker([station['latitude'], station['longitude']], popup=popup_content).add_to(m)

# Marker für die nächstgelegene Ladestation hinzufügen
popup_content = f"<b>Name:</b> {charging_stations[nearest_station_index]['name']}<br><b>Nächste Station</b>"
folium.Marker([nearest_station_lat, nearest_station_long], icon=folium.Icon(color='red'), popup=popup_content).add_to(m)

# Anzeigen der Karte in der Notebook-Zelle
display(m)


hier die Version mit Mapping

In [ ]:
import numpy as np
import folium
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
from IPython.display import display

# Beispiel-Ladestationen mit Längen- und Breitengraden
charging_stations = pd.read_csv("/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/2.landing/ladesauele_id.csv")

# Wählen Sie einen Punkt (z.B. [38.8951, -77.0364]) aus, für den Sie die nächstgelegene Ladestation finden möchten
#query_point = [38.8951, -77.0364]
query_point = [48.456817, 10.029649]


# Koordinaten der Ladestationen
station_coords = np.array([(station["latitude"], station["longitude"]) for station in charging_stations])

# K-NN Algorithmus mit k=1 erstellen
knn = NearestNeighbors(n_neighbors=1)
knn.fit(station_coords)

# Finden der nächstgelegenen Ladestation
distances, nearest_station_index = knn.kneighbors([query_point])

# Index der nächsten Ladestation finden
nearest_station_index = nearest_station_index[0][0]

# Koordinaten der nächsten Ladestation
nearest_station_lat = charging_stations[nearest_station_index]["latitude"]
nearest_station_long = charging_stations[nearest_station_index]["longitude"]

# Berechnen der Distanz zwischen den Koordinaten
query_coord = tuple(query_point)
nearest_station_coord = (nearest_station_lat, nearest_station_long)
distance = geodesic(query_coord, nearest_station_coord).kilometers

# Karte von Deutschland mit Folium erstellen
map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
m = folium.Map(location=map_center, zoom_start=6)

# Marker für Ladestationen hinzufügen
for station in charging_stations:
    popup_content = f"<b>Name:</b> {station['name']}"
    folium.Marker([station['latitude'], station['longitude']], popup=popup_content).add_to(m)

# Marker für die nächstgelegene Ladestation hinzufügen
popup_content = f"<b>Name:</b> {charging_stations[nearest_station_index]['name']}<br><b>Nächste Station</b><br><b>Entfernung:</b> {distance:.2f} km"
folium.Marker([nearest_station_lat, nearest_station_long], icon=folium.Icon(color='red'), popup=popup_content).add_to(m)

# Anzeigen der Karte in der Notebook-Zelle
display(m)


In [ ]:
import pandas as pd
import numpy as np
import folium
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
from IPython.display import display

# Laden der CSV-Datei mit Pandas
data = pd.read_csv("/Users/nowasika/Documents/TechLabs/Team02_ChargeIT/datasets/2.landing/id_ladesaeulenregister.csv")  # Ersetzen Sie "your_data.csv" durch den tatsächlichen Dateinamen

# Koordinaten der Ladestationen aus den Spalten "Latitude" und "Longitude"
charging_stations = data[["Breitengrad", "Längengrad"]].values

# K-NN Algorithmus mit k=2 erstellen (um die nächstgelegenen 2 Stationen zu finden)
knn = NearestNeighbors(n_neighbors=2)
knn.fit(charging_stations)

# Finden der nächstgelegenen Ladestationen für alle Stationen
distances, nearest_station_indices = knn.kneighbors(charging_stations)

# Karte von Deutschland mit Folium erstellen
map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
m = folium.Map(location=map_center, zoom_start=6)

# Marker für Ladestationen hinzufügen
for station_lat, station_long in charging_stations:
    folium.Marker([station_lat, station_long]).add_to(m)

# Marker für die nächstgelegenen Ladestationen hinzufügen
for i, nearest_indices in enumerate(nearest_station_indices):
    nearest_station_lat, nearest_station_long = charging_stations[nearest_indices[1]]
    distance_km = geodesic(charging_stations[i], (nearest_station_lat, nearest_station_long)).kilometers
    popup_content = f"Nächste Station<br>Entfernung: {distance_km:.2f} km"
    folium.Marker([nearest_station_lat, nearest_station_long], icon=folium.Icon(color='red'), popup=popup_content).add_to(m)

# Anzeigen der Karte in der Notebook-Zelle
display(m)


In [ ]:
import pandas as pd
import numpy as np
import folium
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
from IPython.display import display

# Laden der CSV-Datei mit Pandas
data = pd.read_csv("/Users/Takomon/Desktop/IT/techlabs/Team02_ChargeIT/datasets/2.landing/ladesauele_id.csv") 
# Koordinaten der Ladestationen aus den Spalten "Latitude" und "Longitude"
charging_stations = data[["Breitengrad", "Längengrad"]].values

# K-NN Algorithmus mit k=2 erstellen (um die nächstgelegenen 2 Stationen zu finden)
knn = NearestNeighbors(n_neighbors=2)
knn.fit(charging_stations)

# Eingabe von Startpunkt, Zielpunkt und Reichweite durch den Benutzer
start_lat = float(input("Geben Sie die Breitengrad des Startpunkts ein: "))
start_long = float(input("Geben Sie die Längengrad des Startpunkts ein: "))
ziel_lat = float(input("Geben Sie die Breitengrad des Zielpunkts ein: "))
ziel_long = float(input("Geben Sie die Längengrad des Zielpunkts ein: "))
reichweite_km = float(input("Geben Sie die Reichweite Ihres Fahrzeugs in km ein: "))

# Berechnen der Entfernung zwischen Start- und Zielpunkt
start_point = (start_lat, start_long)
ziel_point = (ziel_long, ziel_lat)
gesamtdistanz_km = geodesic(start_point, ziel_point).kilometers

# Finden der nächstgelegenen Ladestationen für Start- und Zielpunkt
distances_start, nearest_start_indices = knn.kneighbors([(start_lat, start_long)])
distances_ziel, nearest_ziel_indices = knn.kneighbors([(ziel_lat, ziel_long)])

# Karte von Deutschland mit Folium erstellen
map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
m = folium.Map(location=map_center, zoom_start=6)

# Marker für Ladestationen hinzufügen
for station_lat, station_long in charging_stations:
    folium.Marker([station_lat, station_long]).add_to(m)

# Marker für die nächstgelegenen Ladestationen hinzufügen
nearest_station_lat_start, nearest_station_long_start = charging_stations[nearest_start_indices[0][1]]
distance_km_start = geodesic(start_point, (nearest_station_lat_start, nearest_station_long_start)).kilometers
popup_content_start = f"Nächste Station am Startpunkt<br>Entfernung: {distance_km_start:.2f} km"
folium.Marker([nearest_station_lat_start, nearest_station_long_start], icon=folium.Icon(color='green'), popup=popup_content_start).add_to(m)

nearest_station_lat_ziel, nearest_station_long_ziel = charging_stations[nearest_ziel_indices[0][1]]
distance_km_ziel = geodesic(ziel_point, (nearest_station_lat_ziel, nearest_station_long_ziel)).kilometers
popup_content_ziel = f"Nächste Station am Zielpunkt<br>Entfernung: {distance_km_ziel:.2f} km"
folium.Marker([nearest_station_lat_ziel, nearest_station_long_ziel], icon=folium.Icon(color='blue'), popup=popup_content_ziel).add_to(m)

# Anzeigen der Karte in der Notebook-Zelle
display(m)

# Ausgabe der Gesamtdistanz und Überprüfung, ob die Reichweite ausreicht
print(f"Gesamtdistanz zwischen Start- und Zielpunkt: {gesamtdistanz_km:.2f} km")
if gesamtdistanz_km <= reichweite_km:
    print("Ihre Reichweite reicht aus, um das Ziel zu erreichen.")
else:
    print("Ihre Reichweite reicht nicht aus, um das Ziel zu erreichen.")


In [ ]:
import pandas as pd
import numpy as np
import folium
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
from IPython.display import display

# Laden der CSV-Datei mit Pandas
data = pd.read_csv("/Users/Takomon/Desktop/IT/techlabs/Team02_ChargeIT/datasets/2.landing/ladesauele_id.csv") 

# Koordinaten der Ladestationen aus den Spalten "Latitude" und "Longitude"
charging_stations = data[["Breitengrad", "Längengrad"]].values

# K-NN Algorithmus mit k=5 erstellen (um die nächstgelegenen 5 Stationen zu finden)
knn = NearestNeighbors(n_neighbors=5)
knn.fit(charging_stations)

# Eingabe von Startpunkt, Zielpunkt und Reichweite durch den Benutzer
start_lat = float(input("Geben Sie die Breitengrad des Startpunkts ein: "))
start_long = float(input("Geben Sie die Längengrad des Startpunkts ein: "))
ziel_lat = float(input("Geben Sie die Breitengrad des Zielpunkts ein: "))
ziel_long = float(input("Geben Sie die Längengrad des Zielpunkts ein: "))
reichweite_km = float(input("Geben Sie die Reichweite Ihres Fahrzeugs in km ein: "))

# Berechnen der Entfernung zwischen Start- und Zielpunkt
start_point = (start_lat, start_long)
ziel_point = (ziel_lat, ziel_long)
gesamtdistanz_km = geodesic(start_point, ziel_point).kilometers

# Finden der nächstgelegenen Ladestationen für Start- und Zielpunkt
distances_start, nearest_start_indices = knn.kneighbors([(start_lat, start_long)])
distances_ziel, nearest_ziel_indices = knn.kneighbors([(ziel_lat, ziel_long)])

# Karte von Deutschland mit Folium erstellen
map_center = [51.1657, 10.4515]  # Zentrum von Deutschland
m = folium.Map(location=map_center, zoom_start=6)

# Marker für Ladestationen hinzufügen
for station_lat, station_long in charging_stations:
    folium.Marker([station_lat, station_long]).add_to(m)

# Marker für die nächstgelegenen Ladestationen auf dem Weg hinzufügen
nearest_station_indices_start = nearest_start_indices[0]
nearest_station_indices_ziel = nearest_ziel_indices[0]

for idx in nearest_station_indices_start:
    if idx in nearest_station_indices_ziel:
        station_lat, station_long = charging_stations[idx]
        distance_km = geodesic(start_point, (station_lat, station_long)).kilometers
        popup_content = f"Nächste Station<br>Entfernung: {distance_km:.2f} km"
        folium.Marker([station_lat, station_long], icon=folium.Icon(color='green'), popup=popup_content).add_to(m)

# Anzeigen der Karte in der Notebook-Zelle
display(m)

# Ausgabe der Gesamtdistanz und Überprüfung, ob die Reichweite ausreicht
print(f"Gesamtdistanz zwischen Start- und Zielpunkt: {gesamtdistanz_km:.2f} km")
if gesamtdistanz_km <= reichweite_km:
    print("Ihre Reichweite reicht aus, um das Ziel zu erreichen.")
else:
    print("Ihre Reichweite reicht nicht aus, um das Ziel zu erreichen.")
